<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.utils import shuffle

Импортируем все, необходимые для работы, библиотеки. Сохраняем данные в переменную **churn** 

Установил значения максимально отображаемых колонок: 20

In [2]:
pd.set_option('display.max_columns', 20)

In [3]:
churn = pd.read_csv('/datasets/Churn.csv')
churn

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [4]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
churn.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


изменил названия колонок приведя их к змеиному регистру

In [6]:
churn = churn.rename(columns={'RowNumber':'row_number', 'CustomerId':'customer_id', 'Surname':'surname',
                             'CreditScore':'credit_score', 'Geography':'geography', 'Gender':'gender',
                             'Age':'age', 'Tenure':'tenure', 'Balance':'balance', 'NumOfProducts':'num_of_products',
                             'HasCrCard':'has_cr_card', 'IsActiveMember':'is_active_member',
                             'EstimatedSalary':'estimated_salary', 'Exited':'exited'})

Пропуски в столбце **Tenure (сколько лет человек является клиентом банка)** заполнил значением 0

In [7]:
churn['tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

In [8]:
churn['tenure'] = churn['tenure'].fillna(churn['tenure'].median())

Вызовем .info повторно для сравнения изменений

In [9]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            10000 non-null  float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Преобразуем категориальные признаки в численные убирая лишние (для обучения модели) признаки **['surname', 'row_number', 'customer_id']**

In [10]:
churn_dummies = pd.get_dummies(churn.drop(['surname', 'row_number', 'customer_id'], axis=1), drop_first=True)
churn_dummies

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


Разделим данные на признаки и на целевой признак

Распределим данные на выборки (обучающая, валидационная, тестовая) в соотношении 3:1:1 (60% : 20% : 20%)

In [11]:
target = churn_dummies['exited']
features = churn_dummies.drop('exited', axis=1)

features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, target, 
                                                                                        test_size=0.40, random_state=333)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test,
                                                                           test_size=0.50, random_state=333)
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)

(6000, 11) (6000,)
(2000, 11) (2000,)
(2000, 11) (2000,)


Убедились в корректности распределения, посмотрев на размеры выборок

Проведена вся необходимая подготовка данных. Приступаем к следующему этапу

## Исследование задачи

Целевой признак может принимать значения 0 или 1. Перед нами стоит задача классификации. Для этой задачи выберем модель из  **RandomForestClassifier** и **DecisionTreeClassifier**

Подберем лучшие по значению **accuracy_score** гипперпараметры для нашей модели. 

In [12]:
%%time
# переменная для сохранения лучшей модели после подбора гиперпараметров для DecisionTreeClassifier
best_decision_tree_model = None 
# переменная для сохранения лучшего значения accuracy
decision_tree_accuracy = 0

# Цикл для перебора гиперпараметра max_depth(от 1 до 20) модели DecisionTreeClassifier
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=333, max_depth=depth)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    if accuracy > decision_tree_accuracy:
        decision_tree_accuracy = accuracy
        best_decision_tree_model = model


CPU times: user 388 ms, sys: 1.33 ms, total: 389 ms
Wall time: 388 ms


In [13]:
print(decision_tree_accuracy, best_decision_tree_model)

0.8535 DecisionTreeClassifier(max_depth=6, random_state=333)


In [14]:
model = DecisionTreeClassifier(random_state=333, max_depth=6)
model.fit(features_train, target_train)
predict = model.predict(features_valid)

print('Показатели лучшей модели DecisionTreeClassifier')
print(f'F1-мера: {f1_score(target_valid, predict)}')
print(f'AUC-ROC: {roc_auc_score(target_valid, predict)}')

Показатели лучшей модели DecisionTreeClassifier
F1-мера: 0.537124802527646
AUC-ROC: 0.691480057605182


In [15]:
%%time
random_forest_accuracy = 0 # переменная для сохранения лучшего значения accuracy 
best_random_forest_model = None # лучшая модель

for est in range(10, 101, 10): # подбираем параметр n_estimators
    for depth in range(1, 20): # подбираем параметр max_depth
        model = RandomForestClassifier(random_state=333, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        predict = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predict)
        if accuracy > random_forest_accuracy:
            random_forest_accuracy = accuracy
            best_random_forest_model = model

CPU times: user 52.5 s, sys: 405 ms, total: 52.9 s
Wall time: 52.9 s


In [16]:
print(random_forest_accuracy, best_random_forest_model)

0.8685 RandomForestClassifier(max_depth=17, n_estimators=50, random_state=333)


***DecisionTreeClassifier*** сильно уступает ***RandomForestClassifier*** по значениям F1-меры и AUC-ROC 

лучшой стала модель ***RandomForestClassifier*** с параметрами: **max_depth=17, n_estimators=50**

Проверим значения ***accuracy, F1-меры и AUC-ROC*** на валидационной выборке

In [17]:
model = RandomForestClassifier(max_depth=17, n_estimators=50, random_state=333)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)

print(f'F1-мера: {f1_score(target_valid, predict_valid)}')
print(f'AUC-ROC: {roc_auc_score(target_valid, predict_valid)}')

F1-мера: 0.5996955859969559
AUC-ROC: 0.725672317990494


Выбранная модель на валидационной выборке показывает результат F1 - меры в 0.599 и AUC-ROC в 0.725

Проверим целевой признак на дисбаланс классов

In [18]:
target.value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Несбалансированность классов почти в 4 раза

Посмотрим какие результаты выдаст модель после балансировки классов

## Борьба с дисбалансом

Для балансировки классов увеличим количество строк меньшего класса в 4 раза 

Сразу же можно попробовать уменьшить выборку убрав значения большего класса в 4 раза

In [19]:
features_zeros = features_train[target_train == 0] # сохраняем признаки целевых признаков равных 0
features_ones = features_train[target_train == 1] # сохраняем признаки целевых признаков равных 1

target_zeros = target_train[target_train == 0] # целевые признаки = 0
target_ones = target_train[target_train == 1] # целевые признаки = 1

# соединяем обучающую выборку увеличив строки с меньшим классом в 4 раза
features_upsampled = pd.concat([features_zeros] + [features_ones] * 4) 
target_upsampled = pd.concat([target_zeros] + [target_ones] * 4)

# смешаем строки с помощью shuffle
features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=333)

# оставляем только 1/4 большего класса
features_downsampled = pd.concat([features_zeros.sample(frac=0.25, random_state=333)] + [features_ones]) 
target_downsampled = pd.concat([target_zeros.sample(frac=0.25, random_state=333)] + [target_ones])

Сохраним увеличение меньшего класса в переменные ***features_upsampled, target_upsampled***, а уменьшение большего класса в переменные ***features_downsampled, target_downsampled***

Проверим показатели после исправления дисбаланса классов

Подберем гиперпараметры снова после балансировки данных

In [20]:
random_forest_accuracy = 0
random_forest_upsampled = None

for est in range(10, 101, 10):
    for depth in range(1, 20):
        model = RandomForestClassifier(random_state=333, max_depth=depth, n_estimators=est, class_weight='balanced')
        model.fit(features_upsampled, target_upsampled)
        predict = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predict)
        if accuracy > random_forest_accuracy:
            random_forest_accuracy = accuracy
            random_forest_upsampled = model

print(random_forest_accuracy, random_forest_upsampled)

0.8545 RandomForestClassifier(class_weight='balanced', max_depth=18, random_state=333)


In [21]:
model_forest_upsampled = random_forest_upsampled
model_forest_upsampled.fit(features_upsampled, target_upsampled)
predict_valid = model_forest_upsampled.predict(features_valid)

print(f'accuracy: {accuracy_score(target_valid, predict_valid)}')
print(f'f1_score: {f1_score(target_valid, predict_valid)}')
print(f'AUC-ROC: {roc_auc_score(target_valid, predict_valid)}')

accuracy: 0.8545
f1_score: 0.6062246278755073
AUC-ROC: 0.7416713537835851


Сохранили параметры модели в переменную ***model_forest_upsampled***

Сбалансировав данные путем дублирования строк меньшего класса изменили гиперпараметры лучшей модели и дали небольшой прирост F1-меры(с 0.599 до 0.606) и увеличение AUC-ROC(с 0.72 до 0.74)

Так же подберем модель для данных с уменьшением большего класса и посмотрим на показатели 

In [22]:
random_forest_accuracy = 0
random_forest_downsamled = None

for est in range(10, 101, 10):
    for depth in range(1, 20):
        model = RandomForestClassifier(random_state=333, max_depth=depth, n_estimators=est, class_weight='balanced')
        model.fit(features_downsampled, target_downsampled)
        predict = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predict)
        if accuracy > random_forest_accuracy:
            random_forest_accuracy = accuracy
            random_forest_downsamled = model

print(random_forest_accuracy, random_forest_downsamled)

0.8045 RandomForestClassifier(class_weight='balanced', max_depth=7, n_estimators=40,
                       random_state=333)


In [23]:
model_forest_downsamled = random_forest_downsamled
model_forest_downsamled.fit(features_downsampled, target_downsampled)
predict_valid = model_forest_downsamled.predict(features_valid)

print(f'accuracy: {accuracy_score(target_valid, predict_valid)}')
print(f'f1_score: {f1_score(target_valid, predict_valid)}')
print(f'AUC-ROC: {roc_auc_score(target_valid, predict_valid)}')

accuracy: 0.8045
f1_score: 0.5964912280701753
AUC-ROC: 0.7699640276653213


Модель с уменьшением выборки увеличило значение AUC-ROC, но уменьшило показатели accuracy и F1-меры.

Оценим показатели и на модели **DecisionTreeClassifier**

In [24]:
%%time
# переменная для сохранения лучшей модели после подбора гиперпараметров для DecisionTreeClassifier
best_tree_upsampled = None 
# переменная для сохранения лучшего значения accuracy
decision_tree_accuracy = 0

# Цикл для перебора гиперпараметра max_depth(от 1 до 20) модели DecisionTreeClassifier
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=333, max_depth=depth, class_weight='balanced')
    model.fit(features_upsampled, target_upsampled)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    if accuracy > decision_tree_accuracy:
        decision_tree_accuracy = accuracy
        best_tree_upsampled = model

CPU times: user 521 ms, sys: 0 ns, total: 521 ms
Wall time: 527 ms


In [25]:
print(decision_tree_accuracy, best_tree_upsampled)

0.792 DecisionTreeClassifier(class_weight='balanced', max_depth=19, random_state=333)


In [26]:
model_tree_upsampled = best_tree_upsampled
model_tree_upsampled.fit(features_upsampled, target_upsampled)
predict = model_tree_upsampled.predict(features_valid)

print('Показатели лучшей модели DecisionTreeClassifier')
print(f'F1-мера: {f1_score(target_valid, predict)}')
print(f'AUC-ROC: {roc_auc_score(target_valid, predict)}')

Показатели лучшей модели DecisionTreeClassifier
F1-мера: 0.48129675810473815
AUC-ROC: 0.6740084429912665


**DecisionTreeClassifier** дала результат хуже чем **RandomForestClassifier**

Для итоговой проверки на тестовой выборке будем использовать именно ее (**model_forest_upsampled**)

с показателями на валидационной выборке:
- accuracy: 0.8545

- F1-мера: 0.6062246278755073

- AUC-ROC: 0.7416713537835851

## Тестирование модели

Протестируем нашу модель на тестовой выборке и посмотрим на результаты

In [27]:
predict_test_upsampled = model_forest_upsampled.predict(features_test)

print(f'accuracy: {accuracy_score(target_test, predict_test_upsampled)}')
print(f'F1-мера: {f1_score(target_test, predict_test_upsampled)}')
print(f'AUC-ROC: {roc_auc_score(target_test, predict_test_upsampled)}')

accuracy: 0.858
F1-мера: 0.6120218579234973
AUC-ROC: 0.7450414937436974


Сравним итоговые результаты с константной моделью

In [28]:
predict = pd.Series(0, index=target_test.index)

print(f'accuracy: {accuracy_score(target_test, predict)}')
print(f'F1-мера: {f1_score(target_test, predict)}')
print(f'AUC-ROC: {roc_auc_score(target_test, predict)}')

accuracy: 0.7985
F1-мера: 0.0
AUC-ROC: 0.5


In [29]:
predict = pd.Series(1, index=target_test.index)

print(f'accuracy: {accuracy_score(target_test, predict)}')
print(f'F1-мера: {f1_score(target_test, predict)}')
print(f'AUC-ROC: {roc_auc_score(target_test, predict)}')

accuracy: 0.2015
F1-мера: 0.3354140657511444
AUC-ROC: 0.5


*Вывод:*  Во время выполнения проекта была выбрана лучшая модель **RandomForestClassifier**, обученная на данных с увеличением меньшего класса и с гипперпараметрам **max_depth=18**. Отброшена **DecisionTreeClassifier.**

Так же был обнаружен дисбаланс классов. Проверены модели до исправления дисбаланса и после. 

Во время тестирования методов исправления несбалансированности классов лучшим, по показателям, оказался способ с увеличением выборки меньшего класса.

После проведения финального тестирования на тестовой выборке, лучшая модель показала результаты:
- accuracy: 0.858
- F1-мера: 0.6120218579234973
- AUC-ROC: 0.7450414937436974

В сравнении с константными дающие в случае заполнения 0 результаты:
- accuracy: 0.7985
- F1-мера: 0.0
- AUC-ROC: 0.5

Модель заполненная 1:
- accuracy: 0.2015
- F1-мера: 0.3354140657511444
- AUC-ROC: 0.5